Por alguna razón, si leemos (read) dos veces el archivo (arx) abierto, la segunda vez nos da un array vacio. 
Asi que mas bien lo leemos y lo cerramos de una.
Al parecer es porque read avanza en el registor, así que si no especificamos el número de bytes o pasos, pus nos leemos todo y quedamos al final. De todas formas por seguridad de cosas que no entiendo, dejemos el close.

In [36]:
arx=open("../ABFiles/Eduardo/C1C31000.DAT", "r")
databin=read(arx)
close(arx)

In [37]:
typeof(databin) # Los datos estan en bytes separados en palabras de 8 bytes.

Array{UInt8,1}

In [38]:
head=databin[1:4]

4-element Array{UInt8,1}:
 0x41
 0x42
 0x46
 0x20

In [39]:
sizeof(Float32)

4

In [40]:
string(head[4]), string(50, base=7), Char(head[4])

("32", "101", ' ')

Lo que descubrió swharden es que el 4 número nos dice si es un abf1 o un abf2. Si el número es un espacio (en ASCII es el número 32) es un abf1. Si es un 50 (ASCII para "2") es un abf2:
 - ascii 32 es espacio -> abf1
 - ascii 50 es 2 -> abf2

In [41]:
convert(Int,databin[4]) #aquí vemos que tipo de archivo es. Curiosamente 0x32 es 50, no 32. ( 3 * 16 +2 )

32

In [42]:
map(Char, head) # and here it is

4-element Array{Char,1}:
 'A'
 'B'
 'F'
 ' '

In [43]:
if( Char(head[4])=='2')
    println( "Tenemos un archivo abf2")
elseif ( Char(head[4])==' ')
    println( "Tenemos un archivo abf1")
else
    println( "no tengo zorra idea de que es esto")
end

Tenemos un archivo abf1


In [44]:
aver=databin[13:16]

4-element Array{UInt8,1}:
 0x16
 0x00
 0x00
 0x00

In [45]:
averviendo=IOBuffer(aver)
read(averviendo, Int32)

22

In [46]:
averviendo=IOBuffer(head)
encabezado=read(averviendo, String)

"ABF "

In [47]:
typeof(encabezado)

String

In [48]:
function readfrombyn(data::Array{UInt8,1}, startbyte::Int, endbyte::Int, typeout::Type)
    # reads byte data for an array, and interprets as typeout, returnint it.
    rango=startbyte:endbyte
    aux=IOBuffer(data[rango])
    result=read(aux, typeout)
    return result
end

readfrombyn (generic function with 1 method)

In [49]:
readfrombyn(databin, 1, 4, String)

"ABF "

In [50]:
aux=IOBuffer(databin[5:8])
read(aux, Float16)

Float16(1638.0)

In [51]:
fFileVersionNumber = readfrombyn(databin, 5,8, Float16)

Float16(1638.0)

In [66]:
nADCPtoLChannelMap = readfrombyn(databin, 379, 410, Array{Int16,16})


ErrorException: The IO stream does not support reading objects of type Array{Int16,16}.

In [59]:
nOperationMode = readfrombyn(databin, 9,10, Int16)
lActualAcqLength = readfrombyn(databin, 11,14, Int32)
lTagSectionPtr = readfrombyn(databin, 45,48,  Int32)
lEpisodesPerRun = readfrombyn(databin, 147, 150, Int32)
lADCResolution = readfrombyn(databin, 252,255, Int32)


8388673

# Encabezado de un ABF1

mmmm tal vez podemos hacer un programa QUE AUTOMATICE esta conversion!

In [60]:
fFileSignature = readfrombyn(f, 1, 4, String )
fFileVersionNumber = readfrombyn(f, 5,8, Float16)
nOperationMode = readfrombyn(f, 9,10, Int16)
lActualAcqLength = readfrombyn(f, 11,14, Int32)
nNumPointsIgnored = readfrombyn(f , 15, 16, Int16)
lActualEpisodes = readfrombyn(f, 17,24, Int32)
lFileStartTime = readfrombyn(f, 25,40 Int32)
lDataSectionPtr = readfrombyn(f, 41,44, Int32)
lTagSectionPtr = readfrombyn(f, 45,48,  Int32)
lNumTagEntries = readfrombyn(f, 49,92, Int32)
lSynchArrayPtr = readfrombyn(f, 93, 96, Int32)
lSynchArraySize = readfrombyn(f, 97,100, Int32)
nDataFormat = readfrombyn(f, 101,120, Int16)
nADCNumChannels = readfrombyn(f, 121,122, Int16)
fADCSampleInterval = readfrombyn(f, 123, 130, Float16)
fSynchTimeUnit = readfrombyn(f, 131,138, Float16)
lNumSamplesPerEpisode = readfrombyn(f, 139,142, Int32)
lPreTriggerSamples = readfrombyn(f, 143, 146, Int32)
lEpisodesPerRun = readfrombyn(f, 147, 150, Int32)
fADCRange = readfrombyn(f, 245,252, Float16)
lADCResolution = readfrombyn(f, 252,255, Int32)
nFileStartMillisecs = readfrombyn(f, 367,368, Int16)
nADCPtoLChannelMap = readfrombyn(f, 379, 410, Array{Int16})
nADCSamplingSeq = readfrombyn(f, "16h", 410)
sADCChannelName = readfrombyn(f, "10s"*16, 442)
sADCUnits = readfrombyn(f, "8s"*16, 602)
fADCProgrammableGain = readfrombyn(f, "16f", 730)
fInstrument
= readfrombyn(f, "16f", 922)
fInstrumentOffset = readfrombyn(f, "16f", 986)
fSignalGain = readfrombyn(f, "16f", 1050)
fSignalOffset = readfrombyn(f, "16f", 1114)
nDigitalEnable = readfrombyn(f, "h", 1436)
nActiveDACChannel = readfrombyn(f, "h", 1440)
nDigitalHolding = readfrombyn(f, "h", 1584)
nDigitalInterEpisode = readfrombyn(f, "h", 1586)
nDigitalValue = readfrombyn(f, "10h", 2588)
lDACFilePtr = readfrombyn(f, "2i", 2048)
lDACFileNumEpisodes = readfrombyn(f, "2i", 2056)
fDACCalibrationFactor = readfrombyn(f, "4f", 2074)
fDACCalibrationOffset = readfrombyn(f, "4f", 2090)
nWaveformEnable = readfrombyn(f, "2h", 2296)
nWaveformSource = readfrombyn(f, "2h", 2300)
nInterEpisodeLevel = readfrombyn(f, "2h", 2304)
nEpochType = readfrombyn(f, "20h", 2308)
fEpochInitLevel = readfrombyn(f, "20f", 2348)
fEpochLevelInc = readfrombyn(f, "20f", 2428)
lEpochInitDuration = readfrombyn(f, "20i", 2508)
lEpochDurationInc = readfrombyn(f, "20i", 2588)
nTelegraphEnable = readfrombyn(f, "16h", 4512)
fTelegraphAdditGain = readfrombyn(f, "16f", 4576)
sProtocolPath = readfrombyn(f, "384s", 4898)

UndefVarError: UndefVarError: f not defined